In [9]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
import mapply
from tqdm import tqdm

tqdm.pandas()

mapply.init(
    n_workers=-1,
    chunk_size=100,
    max_chunks_per_worker=100,
    progressbar=True
)

In [10]:
df = pd.read_csv('../data/data.csv')

In [11]:
stock_codes = np.array(df['StockCode'].unique()).reshape(len(df['StockCode'].unique()),1)
stock_codes
enc = OneHotEncoder()
enc.fit(stock_codes)

OneHotEncoder()

In [12]:
invoice_stock_code = df.groupby('InvoiceNo')['StockCode'].apply(np.array).values
print(invoice_stock_code.shape)
invoice_stock_code = invoice_stock_code[np.array(list(map(len, invoice_stock_code))) > 1]
invoice_stock_code.shape

(25900,)


(20059,)

In [13]:

def get_data(data, enc):
    
    def process_sc(sc, sc_data:list, enc_):
        i = sc_data.index(sc)
        y = enc_.transform(np.array(sc).reshape(1,1)).toarray()
        sc_data.remove(sc)
        x_temp = enc_.transform(np.array(sc_data).reshape(len(sc_data),1))
        x = np.sum(x_temp,axis = 0)
        sc_data.insert(i, sc)
        return x, y

    sc_data_ = list(data.copy())
    return [process_sc(x, sc_data_, enc) for x in data]

In [14]:
invoice_stock_code_serie = pd.Series(invoice_stock_code)
invoice_stock_code_serie_sample = invoice_stock_code_serie.sample(frac=0.1)



In [15]:
get_data_ = lambda x: get_data(x, enc)
result = pd.Series(invoice_stock_code_serie_sample).progress_apply(get_data_)


100%|██████████| 2006/2006 [08:25<00:00,  3.96it/s]


In [73]:
result_numpy = np.concatenate([np.array(x) for x in result.to_numpy()])


In [79]:
split_data = np.split(result_numpy, 2, axis=1)
X = split_data[0]
Y = split_data[1]


In [85]:
with open('../data/x.npy', 'wb') as f:
    np.save(f, X)

with open('../data/y.npy', 'wb') as f:
    np.save(f, Y)